<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/channel_steepness_and_concavity_examples/channel_steepness_and_concavity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing channel steepness and the concavity index with lsdtt-chi-mapping

Last updated by Simon M. Mudd on 09/05/2023

`lsdtt-chi-mapping` is one of the command line tools included in *LSDTopoTools*. This part of *LSDTopoTools* contains many routines for looking at channel steepness indices as well as concavity indices. 

If you want more background you can read these two papers:

* Mudd, S.M., Attal, M., Milodowski, D.T., Grieve, S.W.D., Valters, D.A., 2014. A statistical framework to quantify spatial variation in channel gradients using the integral method of channel profile analysis. Journal of Geophysical Research: Earth Surface 119, 138–152. https://doi.org/10.1002/2013JF002981

* Mudd, S.M., Clubb, F.J., Gailleton, B., Hurst, M.D., 2018. How concave are river channels? Earth Surface Dynamics 6, 505–523. https://doi.org/10.5194/esurf-6-505-2018

If you use this software for your research please cite the aobe papers as appropriate and this software: https://doi.org/10.5281/zenodo.3245040

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

First we install `lsdviztools`. This will take around a minute. It is important you do this before the `condacolab` step. 

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. To get `mamba` we install something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Alternatively we can do this by downloading the mamba installer directly, but this frequently leads to various coding conflicts becasue you need to keep the installer URL up to date. `condacolab` does all that for you so you don't need to worry about it. 

In [ ]:
#%%bash
#MINICONDA_INSTALLER_SCRIPT=Mambaforge-Linux-x86_64.sh
#MINICONDA_PREFIX=/usr/local
#wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh &> /dev/null
#chmod +x $MINICONDA_INSTALLER_SCRIPT
#./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.8.

In [ ]:
!lsdtt-basic-metrics -v

## Download some data

We need to get some data to download. 

For this example we will work nean Xi'an, China (西安) that we studied in this paper:

* Wang, Y.Z., Mudd, S.M., 2021. Evidence for and against landscape transience in the Northern Qinling Mountains, China. Geomorphology 391, 107890. https://doi.org/10.1016/j.geomorph.2021.107890

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key. 

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Xian"
source_name = "COP30"

Xian_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [33.672715066202954, 107.54480443333152], 
                        upper_right_coordinates = [34.16323953210814, 109.4363649228437],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
Xian_DEM.print_parameters()
Xian_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

We can check to see if the file has downloaded

In [ ]:
!ls

## A simple hillshade image

We can run `lsdtt-chi-mapping` with a little interface in `lsdviztools` called the `lsdtt_driver`. Here we are only going to write a hillshade raster.  

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
Dataset_prefix = "Xian"
source_name = "COP30"

lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can plot the image using a `lsdviztools` function. I am printing this to a file and then using the image display option since it allows us to make the figure a bit bigger and in addition you can keep the png image for other uses later. 

You can save the figure as well. This allows you to show it in this notebook in a larger format:

In [ ]:
%%capture  
DataDirectory = "./"
# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code
# WARNING %%capture must be the first line in a code block (you can't even have comments above)
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", 
                                 save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

# Selecting basins

## Automated basin selection

*LSDTopoTools* automates selection of basins. The default way it does this is to grab basins within a window of contributing sizes. Here is an example:

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "minimum_basin_size_pixels" : "10000",
                    "maximum_basin_size_pixels" : "5000000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%%capture             
Base_file = r_prefix
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth", 
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

In this case algorithmic identification is a bit messy for various reasons. This place has a big floodplain near XI'an, that we want to avoid. We also want to avoid all the little basins draining to the south. 

## Choosing basin outlets

One way to get all the basins we want it to make a file with the latitude and longitude of the outlets. We can go into google maps and right click on locations, and then copy these into a latitude and longitude list.

In [ ]:
# Import pandas library
import pandas as pd

data = [ [34.05564682001868, 108.20774244976475],
         [34.023214359957834, 108.5345857094824],
         [34.17775506582951, 107.66257870033067],
         [34.05259180332853, 108.33601473723209],
         [33.996904264107016, 108.75290368815703],
         [34.0171585070696, 108.96363877748513]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

We can use the linux `cat` command to make sure the file is what we expect.

In [ ]:
!cat basin_outlets.csv

## Plotting points using folium

We can plot the basin outlets using a python package called folium

In [ ]:
import folium
import pandas as pd

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

## Get the basins

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()


Now lets look at the channels and basins using `lsdviztools`.
Again, `lsdviztools` does not make interactive plots and is much less flexible than `folium` or `imshow` but it is optimised for publication-ready figures. 

In [ ]:
%%capture
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory = "./"
Dataset_prefix = "Xian"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth", 
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

# Looking at concavity

Let's now run an analysis to test the concavity index. This will take a little while (>20 minutes). You should go and get a coffee or read a book now. 

In order to reduce the compute time I will remove a few basins. 

In [ ]:
# Import pandas library
import pandas as pd

# Select two small catchments
small_data = [ [33.996904264107016, 108.75290368815703],
         [34.0171585070696, 108.96363877748513]]

# Create the pandas DataFrame
sm_df = pd.DataFrame(small_data, columns = ['latitude', 'longitude'])

sm_df.to_csv("two_basin_outlets.csv",index=False)
sm_df.head()

We are going to look at these two small basins (it will save on computational time).
We use the `estimate_best_fit_movern_no_bootstrap` since the bootstrap option is very time consuming and doesn't work as well as the less time consuming disorder method (see https://doi.org/10.5194/esurf-6-505-2018).
We also set how many values of the concavity index we test with the two parameters `delta_movern` and `n_movern`.

In [ ]:
lsdtt_parameters = {"estimate_best_fit_movern_no_bootstrap" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "two_basin_outlets.csv",
                    "delta_movern" : "0.05",
                    "n_movern" : "17"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Okay, lets see what we got!
First, look at the files:

In [ ]:
!ls Xian_COP30*

In [ ]:
import pandas as pd
disorder_df = pd.read_csv("Xian_COP30_UTM_fullstats_disorder_uncert.csv")
disorder_df.head()

In this case the best fit concavity is either 0.4 or 0.45 (the median value). 

We can then look at the disorder plots. 
At the moment we have only a command line python routine for this. 

This will create a very large number of plots. 

In [ ]:
!lsdtt_plotconcavityanalysis -dir . -fname Xian_COP30_UTM -ALL true

This puts all the plots into subdirectories, called `raster_plots`, `summary_plots`, `chi_plots`, and `SA_plots`.

We can use the `!ls` command to see what images are there:

In [ ]:
!ls ./raster_plots

In [ ]:
!ls ./summary_plots

In [ ]:
!ls ./chi_plots

In [ ]:
!ls ./SA_plots

Right, lets have a look at the spatial distribution of the most likely concavity index:

In [ ]:
from IPython.display import display, Image
display(Image(filename="./raster_plots/Xian_COP30_UTM_basins_movern_chi_disorder.png", width=800))

Now lets look at the summary diagram for the disorder metric:

In [ ]:
!ls ./summary_plots

In [ ]:
from IPython.display import display, Image
display(Image(filename="./summary_plots/Xian_COP30_UTM_ABDSG_movern_summary.png", width=800))

In this case the Slope-Area and disorder metrics are giving the same answers, with little uncertainty. The segmented S-A data has very high uncertainty (which is not unusual).

Why don't we have a look at the S-A plots:

In [ ]:
from IPython.display import display, Image
display(Image(filename="./SA_plots/Xian_COP30_UTM_SA_plot_raw_and_binned_basin1.png", width=800))

In [ ]:
from IPython.display import display, Image
display(Image(filename="./SA_plots/Xian_COP30_UTM_SA_plot_raw_and_binned_basin0.png", width=800))

And now lets look at the chi profiles for the best fit concavity:

In [ ]:
from IPython.display import display, Image
display(Image(filename="./chi_plots/chi_profiles0_0.4.png", width=800))

In [ ]:
from IPython.display import display, Image
display(Image(filename="./chi_plots/chi_profiles1_0.45.png", width=800))

If you only wanted to get the disorder metrics and a summary plot you could use this script, which results in far fewer plots:

In [ ]:
!lsdtt_plotconcavityanalysis -dir . -fname Xian_COP30_UTM -disorder true

## Now for exploring chi profiles

So we have two concavity indices, which are 0.4 and 0.45. We can then run a channel steepness analysis using one of these (I will use 0.45). 

We turn on the segmentation algorithm from Mudd et al 2014 (JGR-ES, https://doi.org/10.1002/2013JF002981) with the option ```"print_segmented_M_chi_map_to_csv" : "true"```

In this case we use all of the original basins. This will take a little while to calculate:

In [ ]:
lsdtt_parameters = {"m_over_n" : "0.45",
                    "print_segmented_M_chi_map_to_csv" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "two_basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Okay, great, now we can make some plots. We can do this within python but the command line script is a bit more convinent. 

In [ ]:
!lsdtt_plotchianalysis -dir . -fname Xian_COP30_UTM -all_stacks true -basin_lists "0,1"

These plots end up in a subdirectory called `chi_profile_plots`:

In [ ]:
!ls ./chi_profile_plots

* The `FD` plot is the profiles as a function of flow distance (i.e., a normal channel profile), coloured by $k_{sn}$.
* The `chi` plot is the profiles as a function of chi, coloured by $k_{sn}$.
* The `Sources` plot is the profiles as a function of chi, coloured by the source number.

In [ ]:
from IPython.display import display, Image
display(Image(filename="./chi_profile_plots/Stacked_1_chi_stacked_chi.png", width=800))

With these tools hopefully you can start on your own channel profile analysis!